In [24]:
import os

import numpy as np
import pandas as pd

import cerf


In [10]:
def check_idx(variable, idx, delim):
    """Split expansion variable by delimiter.  Return 'none' as string
    if the requested subcategory is not present.  Otherwise, return the
    name of the requested.

    :param variable:                The variable string to parse.

    :param idx:                     The index number to parse.

    :param delim:                   The variable string delimiter.

    :return:                        'none' if index not present, else
                                    the name.

    """
    try:
        return variable.split(delim)[idx]

    except IndexError:
        return 'none'
    
    

    

In [12]:
f = '/Users/d3y010/projects/cerf/runs/GCAM_CERF_reporting_2019.04.11/GCAM_output_ReEDS_detailed.csv'

scenario = 'GCAMUSA_Reference'
target_yr = 2010



In [20]:
df = pd.read_csv(f, usecols=['Scenario', 'Region-Origin', 'Year', 'Variable', 'Value'])

# rename columns
df.columns = ['scenario', 'primary_zone', 'yr', 'variable', 'value_gw']

# convert any NA values in input to 0
df.fillna(0, inplace=True)

# convert any NA values in input to 0
df.fillna(0, inplace=True)

# extract target scenario and year
df = df.loc[(df['scenario'] == scenario) & (df['yr'] == target_yr)]

# create a catetory variable field
df['category'] = df['variable'].apply(lambda x: check_idx(x, 0, '|'))

# get only electricity capacity values
df = df.loc[(df['category'] == 'Electricity Capacity')]

# split out technology aggregated class (Biomass, Gas, etc.)
df['technology'] = df['variable'].apply(lambda x: check_idx(x, 1, '|'))

# exclude aggregated category that does not have specific technology breakouts
df = df.loc[df['technology'] != 'none']

df['subtech'] = df['variable'].apply(lambda x: check_idx(x, 2, '|'))
df['storage'] = df['variable'].apply(lambda x: check_idx(x, 3, '|'))

df.drop(['variable', 'category'], axis=1, inplace=True)

# convert GW to MW
df['value_mw'] = df['value_gw'] * 1000
df.drop('value_gw', axis=1, inplace=True)


# set cerf_name default
df['cerf_name'] = 'none'

# coal conventional pulverized
df['cerf_name'] = np.where((df['technology'] == 'Coal') &
                           (df['subtech'] == 'Conv Pul') &
                           (df['storage'].isin(('wo CCS', 'w CCS'))),
                           'coal_conv', df['cerf_name'])

# coal igcc without ccs
df['cerf_name'] = np.where((df['technology'] == 'Coal') &
                           (df['subtech'] == 'IGCC') &
                           (df['storage'] == 'wo CCS'),
                           'coal_igcc_wo_ccs', df['cerf_name'])
# coal igcc with ccs
df['cerf_name'] = np.where((df['technology'] == 'Coal') &
                           (df['subtech'] == 'IGCC') &
                           (df['storage'] == 'w CCS'),
                           'coal_igcc_with_ccs', df['cerf_name'])

# gas turbine
df['cerf_name'] = np.where((df['technology'] == 'Gas') &
                           (df['subtech'].isin(('CT', 'ST'))) &
                           (df['storage'].isin(('wo CCS', 'w CCS'))),
                           'gas_turbine', df['cerf_name'])

# gas cc
df['cerf_name'] = np.where((df['technology'] == 'Gas') &
                           (df['subtech'] == 'CC') &
                           (df['storage'].isin(('wo CCS', 'w CCS'))),
                           'gas_cc', df['cerf_name'])

# oil cc
df['cerf_name'] = np.where((df['technology'] == 'Oil') &
                           (df['subtech'] == 'CC') &
                           (df['storage'].isin(('wo CCS', 'w CCS'))),
                           'oil_cc', df['cerf_name'])

# oil turbine
df['cerf_name'] = np.where((df['technology'] == 'Oil') &
                           (df['subtech'] == 'CT') &
                           (df['storage'].isin(('wo CCS', 'w CCS'))),
                           'oil_turbine', df['cerf_name'])

# biomass conventional
df['cerf_name'] = np.where((df['technology'] == 'Biomass') &
                           (df['subtech'] == 'Conv') &
                           (df['storage'].isin(('wo CCS', 'w CCS'))),
                           'biomass', df['cerf_name'])
# biomass igcc without ccs
df['cerf_name'] = np.where((df['technology'] == 'Biomass') &
                           (df['subtech'] == 'IGCC') &
                           (df['storage'] == 'wo CCS'),
                           'biomass_igcc_wo_ccs', df['cerf_name'])

# biomass igcc with ccs
df['cerf_name'] = np.where((df['technology'] == 'Biomass') &
                           (df['subtech'] == 'IGCC') &
                           (df['storage'] == 'w CCS'),
                           'biomass_igcc_with_ccs', df['cerf_name'])
# nuclear
df['cerf_name'] = np.where((df['technology'] == 'Nuclear') &
                           (df['subtech'].isin(('Gen II', 'Gen III'))),
                           'nuclear', df['cerf_name'])

# solar - all combined
df['cerf_name'] = np.where((df['technology'] == 'Solar') &
                           (df['subtech'].isin(('CSP', 'PV'))) &
                           (df['storage'] != 'Dist') &
                           (df['storage'] != 'Non Dist'),
                           'solar_csp', df['cerf_name'])

# wind onshore
df['cerf_name'] = np.where((df['technology'] == 'Wind') &
                           (df['subtech'] == 'Onshore'),
                           'wind_onshore', df['cerf_name'])

# drop none names
df.drop(['technology', 'subtech', 'storage', 'yr'], axis=1, inplace=True)
df = df.loc[df['cerf_name'] != 'none']


In [22]:
df.head()

,scenario,primary_zone,value_mw,cerf_name
6273,GCAMUSA_Reference,AK,0.0,biomass
6274,GCAMUSA_Reference,AL,0.0,biomass
6275,GCAMUSA_Reference,AR,0.0,biomass
6276,GCAMUSA_Reference,AZ,0.0,biomass
6277,GCAMUSA_Reference,CA,0.0,biomass


In [23]:
df.cerf_name.unique()

array(['biomass', 'biomass_igcc_with_ccs', 'biomass_igcc_wo_ccs',
       'coal_conv', 'coal_igcc_with_ccs', 'coal_igcc_wo_ccs', 'gas_cc',
       'gas_turbine', 'nuclear', 'oil_turbine', 'solar_csp',
       'wind_onshore'], dtype=object)

In [48]:
dfx = df.groupby(['primary_zone', 'cerf_name']).sum()

dfx.reset_index(inplace=True)

dfx

,primary_zone,cerf_name,value_mw
0,AK,biomass,0.000000
1,AK,biomass_igcc_with_ccs,0.000000
2,AK,biomass_igcc_wo_ccs,0.000000
3,AK,coal_conv,90.898330
4,AK,coal_igcc_with_ccs,0.000000
...,...,...,...
607,WY,gas_turbine,3.636721
608,WY,nuclear,0.000000
609,WY,oil_turbine,105.084263
610,WY,solar_csp,10.425800


In [30]:
tech_dict = cerf.load_sample_config(2010).get('technology')

tech_dict


{9: {'tech_name': 'biomass',
  'lifetime': 60.0,
  'capacity_factor': 0.6090000000000005,
  'variable_cost_esc_rate': -0.00398993418629034,
  'fuel_esc_rate': 0.0,
  'unit_size': 80,
  'interconnection_cost_per_km': 552,
  'variable_om': 11.68495803744351,
  'heat_rate': 15117.64999999997,
  'fuel_price': 0.0,
  'carbon_capture_rate': 0.0,
  'fuel_co2_content': 0.3035999999999996,
  'discount_rate': 0.05,
  'carbon_esc_rate': 0.0,
  'carbon_tax': 0.0,
  'buffer_in_km': 5,
  'require_pipelines': False,
  'suitability_raster_file': None},
 11: {'tech_name': 'nuclear',
  'lifetime': 60.0,
  'capacity_factor': 0.8999999999999995,
  'variable_cost_esc_rate': -0.00104311614063357,
  'fuel_esc_rate': 0.04639000000000001,
  'unit_size': 1350,
  'interconnection_cost_per_km': 1104,
  'variable_om': 2.098127824402843,
  'heat_rate': 10246.19999999998,
  'fuel_price': 0.7128099999999996,
  'carbon_capture_rate': 0.0,
  'fuel_co2_content': 0.0,
  'discount_rate': 0.05,
  'carbon_esc_rate': 0.0,
  

In [38]:
tech_to_unitsize = {tech_dict[k]['tech_name']: tech_dict[k]['unit_size'] for k in tech_dict.keys()}

tech_to_unitsize


{'biomass': 80, 'nuclear': 1350}

In [52]:
dfx = dfx.loc[dfx['cerf_name'].isin(tech_to_unitsize.keys())].copy()


In [53]:
dfx['unit_size'] = df['cerf_name'].map(tech_to_unitsize)


In [54]:
dfx.head()

,primary_zone,cerf_name,value_mw,unit_size
0,AK,biomass,0.000000,NaN
8,AK,nuclear,0.000000,NaN
12,AL,biomass,68.251397,NaN
20,AL,nuclear,5002.960845,NaN
24,AR,biomass,14.238793,NaN


In [55]:
tech_to_unitsize

{'biomass': 80, 'nuclear': 1350}